##Project Deliverable##

A GitHub repository with a python file (.py) or notebook (.ipynb) with your solution.

## Problem statement##

Telecom companies often have to extract billing data from multiple CSV files generated from
various systems and transform it into a structured format for analysis and revenue reporting.

Thus, there is a need for an automated data pipeline that
can extract billing data from multiple sources and transform it into a structured format for
efficient analysis and revenue reporting

##Guidelings to solving this problems##

● Data Extraction: The data pipeline should be designed to collect data from various sources, including network sensors, equipment sensors, and maintenance records.

● Data Transformation: The collected data must be cleaned and transformed to ensure consistency and quality. This will involve removing duplicates, fixing missing data, and normalizing the data for consistency.

● Data Analysis: The cleaned data will be used to build machine learning models that can predict potential equipment failures and schedule maintenance proactively.


● Data Loading: The resulting data will be stored in a PostgreSQL database.



Data set provided : (https://bit.ly/3YNdO2Y)




The datasets will be in CSV format and will include the following fields:


 1.Equipment sensor data: ID, date, time, sensor reading

 2.Network sensor data: ID, date, time, sensor reading

 3.Maintenance records: ID, date, time, equipment ID, maintenance type


In [1]:
#importing required libraries 
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler

You need to establish connection to the sql DB

In [3]:
POSTGRES_ADDRESS = '22.237.226.11'
POSTGRES_PORT = '5422'
POSTGRES_USERNAME = 'postgresDB'
POSTGRES_PASSWORD = 'password01'
POSTGRES_DBNAME = 'telecommunications_data'

Defining the database engine to be used for parsing 

In [6]:
# Defining the database engine instrumental for data parsing
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
  .format(username=POSTGRES_USERNAME,
     password=POSTGRES_PASSWORD,
     ipaddress=POSTGRES_ADDRESS,
     port=POSTGRES_PORT,
     dbname=POSTGRES_DBNAME))
engine = create_engine(postgres_str)

## Data extraction from data sources on various fields ie Equipment sensor data, Network sensor data, Maintenance records




In [8]:
def extract_data():
    # Load the equipment sensor data
    equipment_df = pd.read_csv('equipment_sensor.csv')

    # Loading the network sensor data
    network_df = pd.read_csv('network_sensor.csv')

    # Load the maintenance records data
    maintenance_df = pd.read_csv('maintenance_records.csv')

    return equipment_df, network_df, maintenance_df

## Data Transformation 


This involve removing duplicates, fixing missing data, normalizing the data for consistency, aggregation , joining and data enrichment.

In [9]:
def transform_data(equipment_df, network_df, maintenance_df):
    # getting rid of the duplicates
    equipment_df.drop_duplicates(inplace=True)
    network_df.drop_duplicates(inplace=True)
    maintenance_df.drop_duplicates(inplace=True)

    # Dealing with missing data
    equipment_df.fillna(method='ffill', inplace=True)
    network_df.fillna(method='ffill', inplace=True)
    maintenance_df.fillna(method='ffill', inplace=True)

    # Normalizing my data
    equipment_df['date_time'] = pd.to_datetime(equipment_df['date'] + ' ' + equipment_df['time'])
    equipment_df.drop(['date', 'time'], axis=1, inplace=True)

    network_df['date_time'] = pd.to_datetime(network_df['date'] + ' ' + network_df['time'])
    network_df.drop(['date', 'time'], axis=1, inplace=True)

    maintenance_df['date_time'] = pd.to_datetime(maintenance_df['date'] + ' ' + maintenance_df['time'])
    maintenance_df.drop(['date', 'time'], axis=1, inplace=True)

    # Aggregating the data
    equipment_summary = equipment_df.groupby('ID').agg({'date_time': ['min', 'max'], 'sensor_reading': ['mean', 'max']})
    equipment_summary.columns = ['first_seen', 'last_seen', 'average_reading', 'max_reading']
    network_summary = network_df.groupby('ID').agg({'date_time': ['min', 'max'], 'sensor_reading': ['mean', 'max']})
    network_summary.columns = ['first_seen', 'last_seen', 'average_reading', 'max_reading']

    # Joining the data from the equipment and network sensor data sets
    sensor_summary = pd.merge(equipment_summary, network_summary, how='outer', left_index=True, right_index=True)
    sensor_summary = sensor_summary.reset_index()
    sensor_summary = sensor_summary.rename(columns={'ID': 'equipment_ID'})

    maintenance_df = maintenance_df[['date_time', 'equipment_ID', 'maintenance_type']]

    return sensor_summary, maintenance_df

#Data Loading 




In [10]:
def load_data(sensor_summary, maintenance_df):
    # using to_sql function to Load the data to PostgreSQL
    sensor_summary.to_sql('sensor_summary', engine, if_exists='replace')
    maintenance_df.to_sql('maintenance_records', engine, if_exists='replace')

##Defining the main function

We will call the main() from within another Python script or from the command line.

In [ ]:
def main():
    equipment_df, network_df, maintenance_df = extract_data()
    sensor_summary, maintenance_df = transform_data(equipment_df, network_df, maintenance_df)
    load_data(sensor_summary, maintenance_df)
    
if __name__ == '__main__':
    main()